In [49]:
import tushare as ts  
import pandas as pd 
# 设置tushare的token，需要到tushare官网申请一个token  
ts.set_token('d6e0ae4ebbc63acf1b4e475f176991bb1d21a16b22380c3c5b5e7da6')  
  
# 初始化pro接口  
pro = ts.pro_api()  
  
# 查询股票代码为'000001.SZ'的股票价格  
df = pro.daily(ts_code='002558.SZ',start_date='20160101')  
df['date'] = pd.to_datetime(df['trade_date'])
df["year"] = pd.to_datetime(df['date']).dt.year.astype(int)
df["month"] = pd.to_datetime(df['date']).dt.month.astype(int)

df['close_open_rate'] = df['close']/df['open']
df["is_up_open"] = ['up' if x > 1 else 'medium' if x == 1 else 'low' for x in df['close_open_rate']]
df["is_up_preclose"] = ['up' if x > 0 else 'medium' if x == 0 else 'low' for x in df['pct_chg']]



In [50]:
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,date,year,month,close_open_rate,is_up_open,is_up_preclose
0,002558.SZ,20231120,12.75,12.96,12.61,12.87,12.72,0.15,1.1792,306207.00,392302.790,2023-11-20,2023,11,1.009412,up,up
1,002558.SZ,20231117,12.69,12.85,12.56,12.72,12.70,0.02,0.1575,217224.80,275308.634,2023-11-17,2023,11,1.002364,up,up
2,002558.SZ,20231116,12.69,12.84,12.62,12.70,12.73,-0.03,-0.2357,223099.88,284544.936,2023-11-16,2023,11,1.000788,up,low
3,002558.SZ,20231115,13.00,13.09,12.68,12.73,12.92,-0.19,-1.4706,294343.35,377200.759,2023-11-15,2023,11,0.979231,low,low
4,002558.SZ,20231114,12.85,12.97,12.80,12.92,12.90,0.02,0.1550,223186.48,287678.362,2023-11-14,2023,11,1.005447,up,up


In [57]:
df['pct_chg'].value_counts(normalize = True)

 0.0000     0.018293
 10.0000    0.004989
 0.3500     0.004435
 0.0500     0.003326
 0.1300     0.002772
              ...   
 0.2786     0.000554
-0.1852     0.000554
-2.6902     0.000554
-0.0953     0.000554
 2.3400     0.000554
Name: pct_chg, Length: 1574, dtype: float64

In [54]:
df.groupby(['year', 'is_up_preclose']).size().div(df.groupby(['year']).size(), level=0)

year  is_up_preclose
2016  low               0.485030
      medium            0.005988
      up                0.508982
2017  low               0.561475
      medium            0.012295
      up                0.426230
2018  low               0.581731
      medium            0.033654
      up                0.384615
2019  low               0.508197
      medium            0.016393
      up                0.475410
2020  low               0.485597
      medium            0.008230
      up                0.506173
2021  low               0.530864
      medium            0.037037
      up                0.432099
2022  low               0.491736
      medium            0.024793
      up                0.483471
2023  low               0.483568
      medium            0.004695
      up                0.511737
dtype: float64

In [62]:
import datetime
from datetime import timedelta  
import yfinance as yf  


def fetch_stock_data(ticker_list, years=5):
    end_date = datetime.datetime.now()  
    start_date = end_date - timedelta(days=years * 365)

    close_data_df = pd.DataFrame()
    high_data_df = pd.DataFrame()
    low_data_df = pd.DataFrame()

    for ticker in ticker_list:
        stock = yf.Ticker(ticker)

        hist_data = stock.history(period='1d', start=start_date, end=end_date)

        close_data = hist_data['Close'].rename(ticker)
        close_data_df = pd.merge(close_data_df, pd.DataFrame(close_data), left_index=True, right_index=True, how='outer')

        high_data = hist_data['High'].rename(ticker)
        high_data_df = pd.merge(high_data_df, pd.DataFrame(high_data), left_index=True, right_index=True, how='outer')

        low_data = hist_data['Low'].rename(ticker)
        low_data_df = pd.merge(low_data_df, pd.DataFrame(low_data), left_index=True, right_index=True, how='outer')

    return close_data_df, high_data_df, low_data_df

# Fetch the data
ticker_list = ['AAPL', 'AMZN', 'MSFT', 'GOOGL', 'META', 'TSLA', 'NVDA', 'ADBE', 'NFLX', 'INTC']
years = 5

close, high, low = fetch_stock_data(ticker_list, years)

In [63]:
close

,AAPL,AMZN,MSFT,GOOGL,META,TSLA,NVDA,ADBE,NFLX,INTC
Date,,,,,,,,,,
2018-11-23 00:00:00-05:00,41.341450,75.102997,97.874969,51.505001,131.729996,21.722000,35.935642,225.559998,258.820007,40.589737
2018-11-26 00:00:00-05:00,41.900547,79.066498,101.103600,52.797001,136.380005,23.066668,37.930683,231.960007,261.429993,41.383404
2018-11-27 00:00:00-05:00,41.809368,79.070999,101.739830,52.613998,135.000000,22.927999,38.099201,232.250000,266.630005,41.924126
2018-11-28 00:00:00-05:00,43.417046,83.887497,105.519241,54.589500,136.759995,23.191334,39.670464,249.210007,282.649994,42.613121
2018-11-29 00:00:00-05:00,43.083511,83.678497,104.636108,54.729000,138.679993,22.744667,39.037857,249.089996,288.750000,41.601425
...,...,...,...,...,...,...,...,...,...,...
2023-11-14 00:00:00-05:00,187.440002,145.800003,369.519989,133.619995,336.309998,237.410004,496.559998,604.330017,448.649994,39.410000
2023-11-15 00:00:00-05:00,188.009995,143.199997,369.670013,134.619995,332.709991,242.839996,488.880005,595.309998,461.940002,40.610001
2023-11-16 00:00:00-05:00,189.710007,142.830002,376.170013,136.929993,334.190002,233.589996,494.799988,602.059998,466.950012,43.349998


In [65]:
def get_high_lows_lookback(high, low, lookback_days):
    lookback_high = high.shift(1).rolling(lookback_days).max()
    lookback_low = low.shift(1).rolling(lookback_days).min()

    return lookback_high, lookback_low

lookback_days = 50
lookback_high, lookback_low = get_high_lows_lookback(high, low, lookback_days)

In [66]:
lookback_high

,AAPL,AMZN,MSFT,GOOGL,META,TSLA,NVDA,ADBE,NFLX,INTC
Date,,,,,,,,,,
2018-11-23 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-26 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-27 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-28 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-11-29 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-11-14 00:00:00-05:00,189.730039,145.860001,369.350356,141.220001,332.329987,278.980011,491.160004,598.140015,453.450012,39.938673
2023-11-15 00:00:00-05:00,188.601529,147.259995,371.196609,141.220001,338.100006,278.980011,498.339996,613.710022,454.079987,39.938673
2023-11-16 00:00:00-05:00,189.500000,147.289993,373.130005,141.220001,338.399994,278.980011,499.600006,613.710022,462.750000,40.840000
